In [1]:
using JuMP, Gurobi, Distributions, Dates;

# Problema

En el contexto de planificacion en muchas areolineas, la asignacion de aviones a rutas siguen la siguiente determinacion:

a) Cuales ciudades servir

b) La frecuencia del servicio entre el Hub y cada una de las otras ciudades

c) Los tiempo de llegada y salida deseados dentro y fuera del Hub para cada ruta por tipo de avion

d) El potencial beneficio asociado de cada ruta a cada diferente tipo de avion en la flota.

El problema de asignacion de rutas  para maximizar el beneficio puede ser formulado como sigue:

\begin{align}
\text{minimize} \qquad & \sum_{i \in I} \sum_{j \in J} P_{ij}X_{ij} \\
 \text{subject to} \quad \quad & \sum_{j \in J}X_{ij} \leq 1 \quad \forall i \in I \quad eq(1)\\
 \qquad \qquad & \sum_{i \in N_{jk}}X_{ij} \leq 1 \quad \forall j\in J, k \in K \quad eq(2)\\
\qquad \qquad & X_{ij} \in \{0,1\} \quad \forall i \in I, j \in J
\end{align}

Definimos la variable de decision:

$X_{ij} = 1$ si ruta i es asignada  a avion j, 0 sino.

Donde I representa el conjunto de rutas, J el conjunto de aviones, K el conjuntos de periodos y $N_{jk}$ subconjuntos de rutas i que podrian utilizar avion j durante periodo k si fuera asignado al avion j.

La restriccion (1) asegura que cada ruta i es asignado al menos a un avion, la restriccion (2) garantiza que cada avion j es asignado a lo mas a una ruta durante cada periodo de tiempo k. Se emplea la desigualdad (1) para asegurar  que una solucion factible existe en el evento que el numero de aviones es menor al numero necesario para asignar todas las rutas.

Las funciones que usamos son las siguientes:

1- loadAndSolveProblem: Con esta funcion se carga el modelo explicado anteriormente, resolviendo asi el problema original

2- loadAndSolveSubProblem: Se usa para resolver la relajacion lagrangiana, donde cada subproblema i consiste es asignar el avion j con el mejor beneficio a la ruta i. Puede resultar que varias rutas sean asigandas al mismo avion en el mismo periodo, por o cual casi nunca generara una solucion factible.

3- genData: Esta funcion se usa para generar la data, esto es, tiempo de salida y tiempo de llegada para cada ruta, $P_{ij}$, $N_{jk}$ y $M_{ij}$.

4- comprobar_fact: Esta rutina comprueba si la solucion obtenida de la relajacion es factible o no para la restriccion (2). Si la solucion Lagrangiana es infactible es porque algun avion fue asignado a mutiples rutas en el mismo periodo k.

5- heuristic_fac: Esta heuristica trata de buscar una solucion factible, reasignando n-1 rutas asignadas a un avion en el mismo periodo a otros aviones que esten disponibles, en caso de no poder, se procede a eliminar esa asignacion y dejar la ruta libre, sin embargo tambien se debe cuidar de satisfacer la restriccion (1) de manera que al final se elimina n-1 rutas que fueron asignadas n veces.



In [2]:
function loadAndSolveProblem(p,K,N,solver)

    #######################################################
    #
    # El problema que estamos considerando es la asignacion de aviones a rutas para
    # maximizar el beneficio obtenido de la programacion de rutas
    # 
    # Basado en el articulo "A Langrangian  Relaxation Approach to Assigning Aircraft to Routes in Hub
    # and Spoke Networks
    #
    #######################################################
    
    # input:
    # solver : CPLEX, GUROBI
    # pij: beneficio de asignar la ruta i al avion j
    # K: Conjunto de periodo de tiempos
    # Njk: conjuntos de rutas i que podrian utilizar el avion j durante el periodo k si es asignado al avion j
    
    # output: 
    # Status: Optima, Infactible, No Acotada
    # Valor Objetivo:
    # Solucion X:
    
    # definimos el nombre del objeto
    prob = Model(solver=solver);

    # calculamos el numero de rutas y aviones
    num_route, num_aircraft = size(p)

    # Xij: 1 si la ruta i es asignada al avion j
    @variable(prob, x[1:num_route,1:num_aircraft] >=0, Bin)
  
    # funcion objetivo: MAX
    @objective(prob, :Max, sum(p[i,j]*x[i,j] for i=1:num_route,j=1:num_aircraft)); 
    
    # res 1
    # Asegura que cada ruta i es asignada a lo menos un avion
    @constraint(prob, res1[i in 1:num_route],sum(x[i,j] for j in 1:num_aircraft) <= 1);
    
    # res 2
    # Asegura que cada avion j es asignada a lo mas una ruta durante un periodo de tiempo k 
    @constraint(prob, res2[j in 1:num_aircraft,k in collect(keys(K))],sum(x[i,j] for i in 1:num_route if in(i,N[j,k])) <= 1);

    # Resolvemos el problema
    status = solve(prob)
                    
    #print(prob)

    return status, getobjectivevalue(prob), getvalue(x)

end

loadAndSolveProblem (generic function with 1 method)

In [3]:
function loadAndSolveSubProblem(p,M,β,K,i,solver)
    
    # input:
    # p array dim 1 x num_aircraft: beneficio de un avion j sobre una ruta dada
    # M array dim num_route x num_aircraft: Conjuntos de periodos de tiempo k durante el cual el avion j podria estar ocupado si este fuera asignado a volar la ruta i
    # β dual array dim num_aircraft x num_periods: Variables duales de la restriccion (res 2)
    # K array dim 1 x num_periods: Conjuntos de periodos de tiempos
    # solver: CPLEX, GUROBI
    
    # output:
    # Status
    # dual
    # objetive
    # X solution
    
    # definimos el nombre del objeto
    prob = Model(solver=solver);

    # calculamos el numero de aviones
    num_aircraft = length(p)

    # Es 1 si se asigna el avion j, sino 0
    @variable(prob, x[1:num_aircraft] >=0)
        
    # Funcion objetivo
    @objective(prob, :Max, sum((p[j]-sum(β[j,k] for k in K if in(k,M[j,i])))*x[j] for j=1:num_aircraft)); 
        
    # res 1
    # Asegura que cada ruta i es asignada a lo menos un avion
    @constraint(prob, res,sum(x[j] for j in 1:num_aircraft) <= 1);

    # resolvemos
    status = solve(prob)
                    
    #print(prob)

    return status, getdual(res), getobjectivevalue(prob), getvalue(x)

end

loadAndSolveSubProblem (generic function with 1 method)

In [4]:
function genData(Avg_route,num_route,num_aircraft)

    #####################################################
    #
    # Esta rutina se usa para generar la data
    #
    #####################################################
    
    # input:
    # Avg_Route: mean route duration (6,8,10) hours
    # num_route: number of routes (50, 75, 100)
    # num_aircraft: number of aircraft.
    
    # output:
    # profit
    # departure
    # arrival
    # K   : set of time periods
    # Njk : set of routes i that would utilize aircraft j during period k if assigned to aircraft j
    # Mji : set of time periods k during which aircraft j would be busy if it were assigned to fly route i
    
    d = DiscreteUniform(0,48) # departure time for each route between 0 and 48 hours. sec 3 
    departure = [floor(Dates.DateTime(Dates.now()+Dates.Hour(rand(d,1)[1])+Dates.Minute(15)), Dates.Minute(15)) for i=1:num_route]

    departure = copy(sort(departure)) # sort
    
    # distribution of route duration
    d = DiscreteUniform(ceil(0.5*Avg_route),ceil(1.5*Avg_route))

    # hours
    aux = [rand(d,1)[1] for i=1:num_route] 
    arrival = [floor(Dates.DateTime(departure[i]+Dates.Hour(aux[i])), Dates.Minute(15)) for i=1:num_route]
    
    # profit per route hour
    d = DiscreteUniform(10,30) 
    profit = [rand(d,1)[1]*aux[i] for i=1:num_route, j=1:num_aircraft]
    
    # set periods time, list array dim 2 (Begins,Ends) occupation time 

    # route  Deperture   Arrival
    # 1      0800        1200
    # 2      1000        1400
    # 3      1100        1800
    # 4      1500        2100
    
    # time peridos constraint (2) example
    #
    # Period     Begins     Ends          Constraints
    # 1          0800       1500      x11 + x12 + x13       <= 1
    # 2          1500       2100                  x13 + x14 <= 1
    
    K = Dict{Int64,Array{Any}}()
    
    K[1] = [departure[1]]
    
    Ea  = 1;
    add = 1;
    for r = 2:num_route
        if departure[r] >= arrival[Ea]
            push!(K[add],departure[r])
            K[add+1] = [K[add][2]];
            add = add + 1;
            Ea = r;
        end
    end
    
    push!(K[add],arrival[Ea])
    
    # all the aircraft are going to have the same routes that can fly for each period
    # this set allows us to build the constraints (2)
    N = Dict{Tuple{Int64,Int64},Array{Int64}}()
    
    for j in 1:num_aircraft
        for k in collect(keys(K))
            N[j,k] = [1];
            for r in 1:num_route
                if (departure[r] >= K[k][1] && departure[r] < K[k][2]) || (arrival[r] > K[k][1] && departure[r] < K[k][2])
                    if ~in(r,N[j,k])
                        push!(N[j,k],r)
                    end
                    if k >= 2 && in(1,N[j,k])
                        shift!(N[j,k])
                    end
                end
            end
        end
    end
       
    # set of time periods k during which aircraft j would be busy if it were assigned to fly route i
    M = Dict{Tuple{Int64,Int64},Array{Int64}}()
    
    for (k,v) in N
        for i in v
            try
                M[k[1],i]
            catch error
                if isa(error, KeyError)
                    M[k[1],i] = [k[2]];
                end
            end
            if ~isempty(M[k[1],i]) && ~in(k[2],M[k[1],i])
                push!(M[k[1],i],k[2])
            end
        end
    end

    
 return departure, arrival, profit, K, N, M
        
end

genData (generic function with 1 method)

In [5]:
function comprobar_fact(X,N)
    
    ########################################
    #
    # Esta rutina comprueba si la solucion obtenida de la restriccion relajada es factible o no.
    # Si la solucion Lagrangiana es infactible es porque algun avion fue asignado a mutiples rutas
    # en el mismo periodo k
    #
    ########################################
    
    # input
    # X: solucion
    # N
    
    # Output
    # conflict : guarda las rutas que hay para un avion j en el periodo k
    # sum_conflict : cuenta el numero de veces que se viola la restriccion (2).
    
    conflict = Dict{Tuple{Int64,Int64},Array{Int64}}((j,k) => [0] for j in 1:num_aircraft, k in 1:num_periods)    
    sum_conflict = []
        
    for j in 1:num_aircraft            
        for k in 1:num_periods
            num_conflict = sum(X[i,j] for i in N[j,k])
            push!(sum_conflict,if  num_conflict > 0  num_conflict - 1 else 0 end) # cuento el numero de violaciones
            #println("aircraft $(j) period $(k) conflic $(if  num_conflict > 0  num_conflict - 1 else 0 end)")
            for i in N[j,k]
                if X[i,j] == 1
                    push!(conflict[j,k],i)
                    if conflict[j,k][1] == 0
                        shift!(conflict[j,k]) # elimino el primer elemento "0"
                    end
                end
            end
        end
    end
     
    return conflict,sum(sum_conflict)
end

comprobar_fact (generic function with 1 method)

In [6]:
function heuristic_fac(conflict,X,N,num_aircraft)

    #########################################
    #
    # Esta heuristica busca una solcion factible (SB)
    # Usa dos forma: reasignar y desasignar
    #
    #########################################
    
    # ejemplo de avion 1 y periodos 1-5:
    # (1, 1) => [1, 3]
    # (1, 2) => [0]
    # (1, 3) => [11, 12, 13]
    # (1, 4) => [11, 15]
    # (1, 5) => [0]
    
    # en el periodo 1 no puede tener dos rutas ya que va a estar ocupado atendiendo solo una ruta en ese periodo
    # el los periodos 3 y 4 hay una ruta en comun, de modo que si muevo la ruta en comun en alguna de ellas ya no la tomare en cuenta en los periodos donde este repetida 
    
    # exite la misma ruta en diferentes periodos de un mismo avion, si la muevo o quito ya no la tomaremos en cuenta
    route_move = []
    
    for (k,v) in conflict
        vr = setdiff(v,route_move)
        v_aux = copy(vr)
        if length(vr) >= 2
            for i in 1:length(vr)-1
                #println(i)
                adis = [sum(X[ii,dis] for ii in N[dis,k[2]]) for dis in 1:num_aircraft] # busco avion disponible
                padis = indmin(adis)
                #println("Posible avion disponible $(padis)")
                if adis[padis] == 0 # al menos algun Avion j esta DISponible, posible asignacion a la ruta i
                
                    avion = find( x->(x == 0), adis) # busco todos los aviones disponibles
                    #println(avion) #
                
                    while (~isempty(avion)) 
                 
                        #padis = avion[1] # elijo el primer elemento
                        # Elijo un avion al azar para intentar salir de optimos locales.
                        padis = sample(avion,1,replace=false)[1]
                
                        ss = indmin(p[v_aux,k[1]] - p[v_aux,padis]) # busco la minima diferencia entre los beneficios de v (rutas)
                        #println("Se puede cambiar la ruta $(v_aux[ss]) al avion $(padis) en el periodo $(k[2])?")
                    
                        # guardo la route que voy a mover o a quitar
                        push!(route_move,v_aux[ss])    
                        
                        # verifico si reasignando puedo mejorar el numero de conflictos actual, es decir, disminuirlos    
                        # primero cuento el numero de conflictos del avion "padis"
                        best_act_conflict = []
                        for all in 1:num_periods
                            push!(best_act_conflict,sum(X[id,padis] for id in N[padis,all]))
                            if best_act_conflict[end] >= 1
                                best_act_conflict[end] = best_act_conflict[end] - 1
                            end
                        end
                        # luego hago el cambio y vuelvo a contar el numero de conflictos
                        sbest_act_conflict = sum(best_act_conflict)
                        #println("conflictos $(sbest_act_conflict)")
                        X[v_aux[ss],k[1]] = 0    # quito un conflicto del avion actual
                        X[v_aux[ss],padis] = 1   # valor propuesto
                        new_act_conflict = []
                        for all in 1:num_periods
                            push!(new_act_conflict,sum(X[id,padis] for id in N[padis,all]))
                            if new_act_conflict[end] >= 1
                                new_act_conflict[end] = new_act_conflict[end] - 1
                            end
                        end
                        snew_act_conflict = sum(new_act_conflict)
                        #println("nuevo conflictos $(snew_act_conflict)")
                        if  snew_act_conflict <= sbest_act_conflict
                            #println("mejora")
                            break # ya asigne a un avion, me salgo, sino sigo con el siguiente avion si hay
                        else
                            #println("no mejora")
                            X[v_aux[ss],padis] = 0 # No asigno la ruta i al avion j
                        end
                
                        # si no asigne nada al avion escogido lo borro de la lista de candidatos
                        #shift!(avion) # borro primer elemento
                        deleteat!(avion, avion .== padis)
                        
                    end
                    
                    #deleteat!(conflict[k], conflict[k] .== [v_aux[ss]])
                    deleteat!(v_aux, v_aux .== [v_aux[ss]])
                else
                    #println("El avion $(padis) no esta disponible")
                    ss = indmin(p[v_aux,k[1]]) # busco el de menor beneficio
                    X[v_aux[ss],k[1]] = 0 # No asigno la ruta i al avion j
                    #deleteat!(conflict[k], conflict[k] .== [v_aux[ss]]) # borro de conflicto
                    deleteat!(v_aux, v_aux .== [v_aux[ss]]) # borro elemento del vector auxiliar
                end
            end
        end
    end
                
    # una ruta no puede ser asignada mas de dos veces (check) 
    route_satisfechas = []
    aircraft_used = []

    for j in 1:num_aircraft
        for i in 1:num_route
            if X[i,j] == 1
                if ~in(j,aircraft_used)
                    push!(aircraft_used,j)
                end
                if ~in(i,route_satisfechas)
                    push!(route_satisfechas,i);
                else
                    println("reasigne la ruta $(i) a otro avion")
                    X[i,j] = 0
                end
            end
        end
    end
    
    conflict,total_conflict = comprobar_fact(X,N);
        
    if total_conflict == 0
        #println("Se construyo una solucion factible")
        SB = sum(p[i,j]*X[i,j] for i in 1:num_route,j in 1:num_aircraft)
    else
        println("No se pudo construir una solucion factible")
    end
    
    return SB,X,conflict,total_conflict,route_satisfechas,aircraft_used
    
end
    

heuristic_fac (generic function with 1 method)

In [13]:
# se define el solver, numero de aviones, numero de rutas y la duracion promedio de las rutas

env = Gurobi.Env()

sub_solver   = GurobiSolver(env,OutputFlag=0)
num_aircraft = 20
num_route    = 50
avg_route    = 8

d,a,p,K,N,M  = genData(avg_route,num_route,num_aircraft);

# se define el numero de periodos
num_periods  = maximum(collect(keys(K)));

#OBS: cada avion se considera como una flota independiente, por ende tendra un diferente precio para cada ruta.


Academic license - for non-commercial use only


In [14]:

# Algoritmo del subgradiente para la relajacion lagrangiana

t0         = time();

# inicializar parametros:

πo         = 2
max_iter   = 1000
max_iter_L = 20
contar_L   = 0
πo_min     = 5e-7

# inicializar solucion:
SB         = 0
best       = 0
XB         = Dict{Tuple{Int64,Int64},Int64}((i,j) => 0 for i in 1:num_route, j in 1:num_aircraft)

β = Dict{Tuple{Int64,Int64},Float64}()

# inicializar β

for j in 1:num_aircraft
    for k in 1:num_periods
        β[j,k] = maximum(p)
    end
end

# inicializar los Bounds

UB = sum(β[j,k] for j in 1:num_aircraft,k in 1:num_periods)
LB = 0

println("iter \tSB \tUB \tGAP")
println("$(0) \t$(SB) \t$(UB) \t$((UB-SB)/UB*100)")

# inicializar G0
G0 = Dict{Tuple{Int64,Int64},Float64}((j,k) => 0 for j in 1:num_aircraft,k in 1:num_periods)

for con in 1:max_iter
    
    # resuelvo los subproblemas independientes
    output = pmap(loadAndSolveSubProblem, 
        [p[i,:] for i = 1:num_route], 
        [M for i = 1:num_route],
        [β for i = 1:num_route],
        [collect(keys(K)) for i=1:num_route],
        [i for i=1:num_route],
        [sub_solver for i = 1:num_route]
        );
    
    # obtenemos solucion
    X = Dict{Tuple{Int64,Int64},Int64}((i,j) => output[i][4][j] for i in 1:num_route, j in 1:num_aircraft)
    
    # calculamos el upper bound
    UB = min(UB,sum((p[i,j] - sum(β[j,k] for k in 1:num_periods if k in M[j,i]))*X[i,j] for i=1:num_route,j in 1:num_aircraft) + sum(β[j,k] for k in 1:num_periods, j in 1:num_aircraft))

    # calculamos el gradiente
    G = Dict{Tuple{Int64,Int64},Float64}()
                    
    # Crowder's damping factor 0<= γ < 1, γ = 0.15  
    for j in 1:num_aircraft
        for k in 1:num_periods
            G[j,k] = 1 - sum(X[i,j] for i in N[j,k]) + 0.25*G0[j,k]
        end
    end
    
    # calculamos la longitud de paso
    #T = πo * (UB - SB)/(sum(G[j,k] for j in 1:num_aircraft, k in 1:num_periods))^2
    T = πo * (UB - SB)/(sum(abs(G[j,k]) for j in 1:num_aircraft, k in 1:num_periods))
                                
    # actualizamos β 
    for j in 1:num_aircraft
        for k in 1:num_periods
            β[j,k] = max(0,β[j,k] - T*G[j,k])
        end
    end
    
    # verificamos si la solucion es factible (por lo general no lo sera)
    conflict,total_conflict = comprobar_fact(X,N);    
    
                                                            
    if total_conflict == 0
        #println("La solucion de la relajacion es factible")
        LB = max(LB,sum(p[i,j]*X[i,j] for i in 1:num_route,j in 1:num_aircraft));
        if LB >= max(SB,LB)
            SB = LB
            XB = X
            best = SB
        end
    else
        #println("La solucion de la relajacion es infactible")
        SB_NEW,X_NEW,conflict,total_conflict,route_satisfechas,aircraft_used = heuristic_fac(conflict,X,N,num_aircraft)
        
                                    
        # asigno rutas no asignadas a aviones disponibles
        act_route = setdiff(1:num_route,route_satisfechas)
        act_aircraft = setdiff(1:num_aircraft,aircraft_used)

        if ~isempty(act_route) && ~isempty(act_aircraft)
            for j in act_aircraft
                if ~isempty(act_route)
                    route_satisfechas = []
                    time_end = Dates.DateTime(d[act_route[1]]-Dates.Hour(5))
                    for i in act_route
                        if d[i] >= time_end
                            X_NEW[i,j] = 1
                            push!(route_satisfechas,i)
                            time_end = a[i]
                        end
                    end
                    act_route = setdiff(copy(act_route),route_satisfechas)
                end
            end
        
        # verifico si no hay conflictos
        conflict,total_conflict = comprobar_fact(X_NEW,N);
                                       
        # volvemos a calcular todo para actualiar X_NEW
        SB_NEW,X_NEW,conflict,total_conflict,route_satisfechas,aircraft_used = heuristic_fac(conflict,X_NEW,N,num_aircraft)

        end
                                    
        if total_conflict >= 1
            println("Infactible solucion")
            break
        end
        if SB_NEW >= max(SB,LB)
            SB = SB_NEW
            XB = X_NEW
            best = SB
        end
    end

    # verifico si no hay mejora en la mejor solucion factible encontrada
    if best == SB
        contar_L = contar_L + 1;
        if contar_L == max_iter_L
            contar_L = 0
            πo = πo/2
        end
    else
        contar_L = 0;
    end                                

    if mod(con,20) == 0
        println("$(con) \t$(SB) \t$(UB) \t$((UB-SB)/UB*100)")
    end
    
    # actualizamos G0
    G0 = Dict{Tuple{Int64,Int64},Float64}((j,k) => G[j,k] for j in 1:num_aircraft,k in 1:num_periods)

    if πo_min > πo || (UB-SB)/UB*100 < 1e-2 || UB-SB < 1
        println("$(con) \t$(SB) \t$(UB) \t$((UB-SB)/UB*100)")
        break
    end
    
end
                            
timefinal = time() - t0
println("Elapsed Time: ", timefinal, " seconds")



iter 	SB 	UB 	GAP
0 	0 	43200.0 	100.0
20 	9991 	11812.0 	15.416525567219777
40 	10054 	11094.883731641046 	9.381655155813778
60 	10308 	10911.568755246572 	5.5314571972646895
80 	10847 	10888.696640293283 	0.3829350901281053
99 	10886 	10887.052353838635 	0.009666104326797443
Elapsed Time: 5.665923833847046 seconds


In [12]:
# solucion con el problema original

@time output = pmap(loadAndSolveProblem, 
        [p for i = 1:1], 
        [K for i = 1:1],
        [N for i = 1:1],
        [sub_solver for i = 1:1]
        )


  2.478938 seconds (633.48 k allocations: 33.697 MiB, 0.51% gc time)


1-element Array{Tuple{Symbol,Float64,Array{Float64,2}},1}:
 (:Optimal, 11749.0, [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [47]:

# Imprimir solucion con el algoritmo del subgradiente

route_satisfechas = []
aircraft_used = []

for j in 1:num_aircraft
    println("Aircraft $(j)")
    for i in 1:num_route
        if XB[i,j] == 1
            if ~in(j,aircraft_used)
                push!(aircraft_used,j)
            end
            if ~in(i,route_satisfechas)
                push!(route_satisfechas,i);
            else
                println("Error la misma ruta asignada dos veces")
                break
            end
            println("route $(i) - \t$(d[i]) - $(a[i])")
        end
    end
end

println("satisfied route: $(length(route_satisfechas)/num_route*100)%")
println("Aircraft used: $(length(aircraft_used)/num_aircraft*100)%")


Aircraft 1
route 26 - 	2018-12-23T02:15:00 - 2018-12-23T11:15:00
route 71 - 	2018-12-23T23:15:00 - 2018-12-24T09:15:00
Aircraft 2
route 9 - 	2018-12-22T16:15:00 - 2018-12-22T20:15:00
route 17 - 	2018-12-22T23:15:00 - 2018-12-23T11:15:00
route 75 - 	2018-12-24T02:15:00 - 2018-12-24T14:15:00
Aircraft 3
route 6 - 	2018-12-22T15:15:00 - 2018-12-22T22:15:00
route 42 - 	2018-12-23T08:15:00 - 2018-12-23T15:15:00
route 56 - 	2018-12-23T19:15:00 - 2018-12-23T23:15:00
route 74 - 	2018-12-24T02:15:00 - 2018-12-24T11:15:00
Aircraft 4
route 29 - 	2018-12-23T03:15:00 - 2018-12-23T11:15:00
route 51 - 	2018-12-23T14:15:00 - 2018-12-24T01:15:00
route 84 - 	2018-12-24T06:15:00 - 2018-12-24T16:15:00
Aircraft 5
route 2 - 	2018-12-22T13:15:00 - 2018-12-22T18:15:00
route 37 - 	2018-12-23T06:15:00 - 2018-12-23T13:15:00
route 64 - 	2018-12-23T22:15:00 - 2018-12-24T05:15:00
Aircraft 6
route 3 - 	2018-12-22T13:15:00 - 2018-12-22T21:15:00
route 32 - 	2018-12-23T05:15:00 - 2018-12-23T11:15:00
route 62 - 	2018-12-

In [38]:
Xoriginal = Dict{Tuple{Int64,Int64},Int64}((i,j) => output[1][3][i,j] for i in 1:num_route, j in 1:num_aircraft)

# Imprimir solucion del problema original

route_satisfechas = []
aircraft_used = []

for j in 1:num_aircraft
    println("Aircraft $(j)")
    for i in 1:num_route
        if Xoriginal[i,j] == 1
            if ~in(j,aircraft_used)
                push!(aircraft_used,j)
            end
            if ~in(i,route_satisfechas)
                push!(route_satisfechas,i);
            else
                println("Error la misma ruta asignada dos veces")
                break
            end
            println("route $(i) - \t$(d[i]) - $(a[i])")
        end
    end
end

println("satisfied route: $(length(route_satisfechas)/num_route*100)%")
println("Aircraft used: $(length(aircraft_used)/num_aircraft*100)%")


Aircraft 1
route 7 - 	2018-12-22T16:45:00 - 2018-12-22T21:45:00
route 15 - 	2018-12-23T02:45:00 - 2018-12-23T10:45:00
route 35 - 	2018-12-23T23:45:00 - 2018-12-24T10:45:00
Aircraft 2
route 4 - 	2018-12-22T11:45:00 - 2018-12-22T17:45:00
route 18 - 	2018-12-23T05:45:00 - 2018-12-23T17:45:00
route 39 - 	2018-12-24T01:45:00 - 2018-12-24T09:45:00
Aircraft 3
route 1 - 	2018-12-22T05:45:00 - 2018-12-22T10:45:00
route 3 - 	2018-12-22T11:45:00 - 2018-12-22T17:45:00
route 12 - 	2018-12-23T00:45:00 - 2018-12-23T10:45:00
route 33 - 	2018-12-23T21:45:00 - 2018-12-24T08:45:00
Aircraft 4
route 6 - 	2018-12-22T13:45:00 - 2018-12-22T17:45:00
route 11 - 	2018-12-22T23:45:00 - 2018-12-23T08:45:00
route 25 - 	2018-12-23T09:45:00 - 2018-12-23T21:45:00
route 38 - 	2018-12-24T00:45:00 - 2018-12-24T06:45:00
Aircraft 5
route 5 - 	2018-12-22T12:45:00 - 2018-12-22T22:45:00
route 14 - 	2018-12-23T01:45:00 - 2018-12-23T08:45:00
route 24 - 	2018-12-23T08:45:00 - 2018-12-23T12:45:00
route 32 - 	2018-12-23T20:45:00 -